In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import nibabel as nib
import cv2 as cv
import os
from IPython.display import Image
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Conv2D,Conv2DTranspose,MaxPool2D,Concatenate, Activation, BatchNormalization,concatenate ,Input
from tensorflow.keras.models import Model
from sklearn.metrics import multilabel_confusion_matrix
from model import classification_model,classification_resnet,classification_resnet

# classification training and test result

In [4]:
model=classification_model((256,256,3))
model2=classification_resnet((256,256,3))

In [ ]:
# model optimizer and loss 
model.compile(optimizer='adam',loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])
model2.compile(optimizer='adam',loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [ ]:
# data loading
x_train=np.load('classification_image.npy')
label=np.load("classification_label.npy")

In [ ]:
# callback=tf.keras.callbacks.EarlyStopping(monitor='loss',patience=3,min_delta=0.001)
history=model.fit(x_train,label,batch_size=20,epochs=100,callbacks=[callback])

In [ ]:
# callback=tf.keras.callbacks.EarlyStopping(monitor='loss',patience=3,min_delta=0.001)
history2=model2.fit(x_train,label,batch_size=20,epochs=50,callbacks=[callback])

**result and plot**

In [ ]:
plt.subplot(121)
plt.plot(history.history['accuracy'])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.subplot(122)
plt.plot(history.history['loss'])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

In [ ]:
plt.subplot(121)
plt.plot(history2.history['accuracy'])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.subplot(122)
plt.plot(history2.history['loss'])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

In [ ]:
y_pred=model(x_train)
pred=np.argmax(y_pred,axis=-1)
y_pred2=model2(x_train)
pred2=np.argmax(y_pred2,axis=-1)
label=np.load("classification_label.npy")
actual=np.argmax(label,axis=-1)
result=multilabel_confusion_matrix(actual,pred)
result2=mutilabel_confusion_matrix(actual,pred2)
print(result)
print(result2)

# Segmentation

In [ ]:
seg_model=segmentation_model((512,512,3))
seg_model.summary()

In [ ]:
images=np.load("covid-segmentation/images_medseg.npy").astype(np.float32)
mask=np.load("covid-segmentation/masks_medseg.npy").astype(np.float32)

In [ ]:
timage=[]
for image in images:
    new_image=cv.cvtColor(image,cv.COLOR_GRAY2RGB)
    timage.append(new_image)
timages=np.array(timage)
fimages=cv.normalize(timages,None,0,255,cv.NORM_MINMAX,cv.CV_32F).astype(np.int8)

In [ ]:
seg_model.compile(optimizer=tf.optimizers.RMSprop(learning_rate=0.14),loss='binary_crossentropy',metrics=[tf.keras.metrics.MeanIoU(num_classes=4)])
checkpoint_dir = './training_checkpoints_new'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")
callback2=tf.keras.callbacks.ModelCheckpoint(monitor="loss",filepath=checkpoint_prefix,save_weights_only=True)

In [ ]:
seg_history=seg_model.fit(fimages,mask,batch_size=1,epochs=100,verbose=1,callbacks=[callback2])

In [ ]:
plt.subplot(121)
plt.plot(seg_history.history['mean_io_u'])
plt.title("mean io_u")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.subplot(122)
plt.plot(seg_history.history['loss'])
plt.title("loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.savefig('output.jpg')
plt.show()

In [ ]:
checkpoint_dir = './training_checkpoints_new/'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")
seg_model.load_weights(checkpoint_prefix)

**spreading of disease and dice coffiecient**

In [ ]:
def spreading(image):
    carry=np.unique(image,return_counts=True)
    dict={0:0,1:0,2:0,3:0}
    for i in range(len(carry[0])):
        dict[carry[0][i]]=carry[1][i]
    return dict[0]*100/(dict[1]+dict[0]+dict[2])

def dice(im1, im2):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / (im1.sum() + im2.sum())